In [7]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [8]:
from langchain_core.documents import Document

doc1 = Document(
        page_content="Virat Kohli is one of the most successful and consistent batsmen in IPL history. Known for his aggressive batting style and fitness, he has led the Royal Challengers Bangalore in multiple seasons.",
        metadata={"team": "Royal Challengers Bangalore"}
    )
doc2 = Document(
        page_content="Rohit Sharma is the most successful captain in IPL history, leading Mumbai Indians to five titles. He's known for his calm demeanor and ability to play big innings under pressure.",
        metadata={"team": "Mumbai Indians"}
    )
doc3 = Document(
        page_content="MS Dhoni, famously known as Captain Cool, has led Chennai Super Kings to multiple IPL titles. His finishing skills, wicketkeeping, and leadership are legendary.",
        metadata={"team": "Chennai Super Kings"}
    )
doc4 = Document(
        page_content="Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over expertise.",
        metadata={"team": "Mumbai Indians"}
    )
doc5 = Document(
        page_content="Ravindra Jadeja is a dynamic all-rounder who contributes with both bat and ball. Representing Chennai Super Kings, his quick fielding and match-winning performances make him a key player.",
        metadata={"team": "Chennai Super Kings"}
    )

In [9]:
docs = [doc1, doc2, doc3, doc4, doc5]

In [10]:
vector_store = Chroma(
    embedding_function=embedding,
    persist_directory='my_chroma_db',
    collection_name='sample'
)

In [ ]:
#add documents
vector_store.add_documents(docs)

['4ce8d4a3-76b7-4c72-852d-d79aa7a017f7',
 'abe8c264-ef15-4cf5-864e-eee4d1cad983',
 'dfcd285e-0e5e-4b1c-a7fd-c3db983a1648',
 '319e75d6-7d08-471f-b9c5-dd3b3d66c7e7',
 'f36a585c-f964-4289-8850-1ecfb2ff5b08']

In [12]:
#view documents
vector_store.get(include=['embeddings','documents', 'metadatas'])

{'ids': ['4ce8d4a3-76b7-4c72-852d-d79aa7a017f7',
  'abe8c264-ef15-4cf5-864e-eee4d1cad983',
  'dfcd285e-0e5e-4b1c-a7fd-c3db983a1648',
  '319e75d6-7d08-471f-b9c5-dd3b3d66c7e7',
  'f36a585c-f964-4289-8850-1ecfb2ff5b08'],
 'embeddings': array([[ 0.00994725,  0.06914336, -0.05147117, ..., -0.03543336,
          0.01284807,  0.01248293],
        [ 0.00127744,  0.03129855, -0.02375379, ..., -0.00518363,
         -0.03280612,  0.02737716],
        [-0.10265914,  0.02650812,  0.02271501, ..., -0.03359743,
         -0.07984945, -0.01507708],
        [ 0.02123395, -0.02468543, -0.04494375, ..., -0.10995815,
          0.00572563,  0.09915379],
        [ 0.01873984,  0.04382849, -0.04304257, ..., -0.07801617,
         -0.07840681, -0.00304191]], shape=(5, 384)),
 'documents': ['Virat Kohli is one of the most successful and consistent batsmen in IPL history. Known for his aggressive batting style and fitness, he has led the Royal Challengers Bangalore in multiple seasons.',
  "Rohit Sharma is the mo

In [13]:
# search documents
vector_store.similarity_search(
    query='Who among these are a bowler?',
    k=2
)

[Document(id='319e75d6-7d08-471f-b9c5-dd3b3d66c7e7', metadata={'team': 'Mumbai Indians'}, page_content='Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over expertise.'),
 Document(id='abe8c264-ef15-4cf5-864e-eee4d1cad983', metadata={'team': 'Mumbai Indians'}, page_content="Rohit Sharma is the most successful captain in IPL history, leading Mumbai Indians to five titles. He's known for his calm demeanor and ability to play big innings under pressure.")]

In [14]:
# search with similarity score
vector_store.similarity_search_with_score(
    query='Who among these are a bowler?',
    k=2
)

[(Document(id='319e75d6-7d08-471f-b9c5-dd3b3d66c7e7', metadata={'team': 'Mumbai Indians'}, page_content='Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over expertise.'),
  0.9693600535392761),
 (Document(id='abe8c264-ef15-4cf5-864e-eee4d1cad983', metadata={'team': 'Mumbai Indians'}, page_content="Rohit Sharma is the most successful captain in IPL history, leading Mumbai Indians to five titles. He's known for his calm demeanor and ability to play big innings under pressure."),
  1.1493446826934814)]

In [15]:
# meta-data filtering
vector_store.similarity_search_with_score(
    query="",
    filter={"team": "Chennai Super Kings"}
)

[(Document(id='dfcd285e-0e5e-4b1c-a7fd-c3db983a1648', metadata={'team': 'Chennai Super Kings'}, page_content='MS Dhoni, famously known as Captain Cool, has led Chennai Super Kings to multiple IPL titles. His finishing skills, wicketkeeping, and leadership are legendary.'),
  1.8436003923416138),
 (Document(id='f36a585c-f964-4289-8850-1ecfb2ff5b08', metadata={'team': 'Chennai Super Kings'}, page_content='Ravindra Jadeja is a dynamic all-rounder who contributes with both bat and ball. Representing Chennai Super Kings, his quick fielding and match-winning performances make him a key player.'),
  1.8909372091293335)]

In [16]:
# update documents
updated_doc1 = Document(
    page_content="Virat Kohli, the former captain of Royal Challengers Bangalore (RCB), is renowned for his aggressive leadership and consistent batting performances. He holds the record for the most runs in IPL history, including multiple centuries in a single season. Despite RCB not winning an IPL title under his captaincy, Kohli's passion and fitness set a benchmark for the league. His ability to chase targets and anchor innings has made him one of the most dependable players in T20 cricket.",
    metadata={"team": "Royal Challengers Bangalore"}
)

vector_store.update_document(document_id='09a39dc6-3ba6-4ea7-927e-fdda591da5e4', document=updated_doc1)


In [17]:
# view documents
vector_store.get(include=['embeddings','documents', 'metadatas'])

{'ids': ['4ce8d4a3-76b7-4c72-852d-d79aa7a017f7',
  'abe8c264-ef15-4cf5-864e-eee4d1cad983',
  'dfcd285e-0e5e-4b1c-a7fd-c3db983a1648',
  '319e75d6-7d08-471f-b9c5-dd3b3d66c7e7',
  'f36a585c-f964-4289-8850-1ecfb2ff5b08'],
 'embeddings': array([[ 0.00994725,  0.06914336, -0.05147117, ..., -0.03543336,
          0.01284807,  0.01248293],
        [ 0.00127744,  0.03129855, -0.02375379, ..., -0.00518363,
         -0.03280612,  0.02737716],
        [-0.10265914,  0.02650812,  0.02271501, ..., -0.03359743,
         -0.07984945, -0.01507708],
        [ 0.02123395, -0.02468543, -0.04494375, ..., -0.10995815,
          0.00572563,  0.09915379],
        [ 0.01873984,  0.04382849, -0.04304257, ..., -0.07801617,
         -0.07840681, -0.00304191]], shape=(5, 384)),
 'documents': ['Virat Kohli is one of the most successful and consistent batsmen in IPL history. Known for his aggressive batting style and fitness, he has led the Royal Challengers Bangalore in multiple seasons.',
  "Rohit Sharma is the mo

In [18]:
# delete document
vector_store.delete(ids=['09a39dc6-3ba6-4ea7-927e-fdda591da5e4'])

In [19]:
# view documents
vector_store.get(include=['embeddings','documents', 'metadatas'])

{'ids': ['4ce8d4a3-76b7-4c72-852d-d79aa7a017f7',
  'abe8c264-ef15-4cf5-864e-eee4d1cad983',
  'dfcd285e-0e5e-4b1c-a7fd-c3db983a1648',
  '319e75d6-7d08-471f-b9c5-dd3b3d66c7e7',
  'f36a585c-f964-4289-8850-1ecfb2ff5b08'],
 'embeddings': array([[ 0.00994725,  0.06914336, -0.05147117, ..., -0.03543336,
          0.01284807,  0.01248293],
        [ 0.00127744,  0.03129855, -0.02375379, ..., -0.00518363,
         -0.03280612,  0.02737716],
        [-0.10265914,  0.02650812,  0.02271501, ..., -0.03359743,
         -0.07984945, -0.01507708],
        [ 0.02123395, -0.02468543, -0.04494375, ..., -0.10995815,
          0.00572563,  0.09915379],
        [ 0.01873984,  0.04382849, -0.04304257, ..., -0.07801617,
         -0.07840681, -0.00304191]], shape=(5, 384)),
 'documents': ['Virat Kohli is one of the most successful and consistent batsmen in IPL history. Known for his aggressive batting style and fitness, he has led the Royal Challengers Bangalore in multiple seasons.',
  "Rohit Sharma is the mo